In [1]:
import os
from google.oauth2.service_account import Credentials

# Get the path from the environment variable
json_path = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")

if json_path:
    # Authenticate with the Google API
    credentials = Credentials.from_service_account_file(json_path)
    print("Credentials loaded successfully!")
else:
    print("Environment variable not set. Please set GOOGLE_APPLICATION_CREDENTIALS.")

print(json_path)

Environment variable not set. Please set GOOGLE_APPLICATION_CREDENTIALS.
None
